In [ ]:
%pip install python-dotenv -q
%pip install SPARQLWrapper -q
%pip install pandas -q
%pip install neo4j -q

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.addCustomHttpHeader('User-Agent', 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)')


# Exploring data

In [17]:
# Fetch all romain deities with their domains
#
# P31 instance of
# Q11688446 roman deity
# P2925 domain of saint or deity
sparql.setQuery("""
SELECT ?romainDeity ?romainDeityLabel ?romainDeityDescription (group_concat(?domainOfSaint;separator=";") as ?domainsOfSaint) {
  ?romainDeity wdt:P31 wd:Q11688446;
  wdt:P2925  ?domainOfSaint .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?romainDeity ?romainDeityLabel ?romainDeityDescription
LIMIT 5
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for row in results["results"]["bindings"]:
    title = row["romainDeityLabel"]["value"]
    if ('romainDeityDescription' in row):
      title += "\t" + row['romainDeityDescription']['value']
    
    print(title)    
    domains = row['domainsOfSaint']['value'].split(';')    
    if (domains):
      for domain in domains:
          print("\t" + domain)
    

Mars	Roman god of war, guardian of agriculture
	http://www.wikidata.org/entity/Q198
	http://www.wikidata.org/entity/Q11451
Ceres	Roman goddess of agriculture
	http://www.wikidata.org/entity/Q11451
	http://www.wikidata.org/entity/Q964401
	http://www.wikidata.org/entity/Q1956046
	http://www.wikidata.org/entity/Q2995529
Gaia	Greek primordial deity, the personification of the Earth
	http://www.wikidata.org/entity/Q964401
Juno	ancient Roman goddess
	http://www.wikidata.org/entity/Q8445
	http://www.wikidata.org/entity/Q42008
	http://www.wikidata.org/entity/Q866081
	http://www.wikidata.org/entity/Q14819852
Minerva	Roman goddess of wisdom and sponsor of arts, trade, and defense
	http://www.wikidata.org/entity/Q37226
	http://www.wikidata.org/entity/Q49757
	http://www.wikidata.org/entity/Q1294787
	http://www.wikidata.org/entity/Q43109919


In [7]:
# Distinct domains of saint or deity (P2925) values
#
# P2925 domain of saint or deity
sparql.setQuery("""
SELECT DISTINCT ?deity ?deityLabel
WHERE 
{
  ?deity wdt:P31 wd:Q178885 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  # FILTER ( ?domain in ( wd:Q178885 ) )
  
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# print(results)

for row in results["results"]["bindings"]:
    if ('deityLabel' not in row):
      continue
    
    title = row['deityLabel']['value']
    print(title)


Q167008
Taionjin
Ahura Mazda
God the Father
Zhongli Quan
Abraxas
Q402052
Adramelech
Apaliunas
Agon
Q637803
Q660031
Ogma
Devi
Q793294
Palici
Q891215
Baal
Iat
Lü Dongbin
Ji Gong
Phra Phrom
Ansa
Li Jing
Tara
Asura
Omoikane
Billiken
Qingshui
Ilmarinen
Lono
Han Xiang
Beelzebub
Belphegor
Angra Mainyu
Dōsojin
Nezha
Abellio
Lieu Hanh
Ḫepat
Ekeko
Atarrabi
Ugajin
Bathala
Sarruma
Shinigami
Popiel
Hubal
Maya death gods
Lan Caihe
Zhurong
Immortal Woman He
Yngvi
Väinämöinen
Hadad
Soranus
Maximón
Ataegina
Šauška
Antenociticus
Fascinus
Anshar
Istanu
Kothar-wa-Khasis
Aide
Ennead
Ullikummi
Souls of Pe and Nekhen
Lelwani
Ame-no-tajikarao
Juok
Seven Lucky Gods
Kitchen god
deva
Aphrodite Urania
Nunbarsegunu
Upelluri
Barrex
Uttu
Agasaya
Mahamayuri
Kubaba
L727946
Elder Zhang Guo
Zhang Yazi
Mikogami
Penarddun
Trailokyavijaya
Paraclete
Bamapana
King Father of the East
Shahmaran
Aillen
Ai apaec
Theispas
Mór Muman
Jesus Christ
Juracán
Pizamar
Abgal


In [11]:
# Count domains of saint or deity (P2925) values
#
# P2925 domain of saint or deity
sparql.setQuery("""
SELECT (count(DISTINCT ?value) as ?count)
WHERE 
{
  ?item wdt:P2925/wdt:P1629*/wd:Q178885* ?value
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
""")

sparql.setReturnFormat(JSON)
domainsOfSaintOrDeity = sparql.query().convert()

print(domainsOfSaintOrDeity['results']['bindings'][0]['count']['value'])

738


# Fetch all war deities


In [13]:
# P31 instance of
# Q41863069 war deity
sparql.setQuery("""
SELECT ?wargod ?wargodLabel (group_concat(?domainOfSaint;separator=";") as ?domainsOfSaint) {
  ?wargod wdt:P31 wd:Q41863069;
  wdt:P2925  ?domainOfSaint
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?wargod ?wargodLabel
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print_limit = 10
for row in results["results"]["bindings"][:print_limit]:
  print(row["wargodLabel"]["value"] + ": " + row["wargod"]['value'])  
  
  domains = row['domainsOfSaint']['value'].split(';')    
  if (domains):
    for domain in domains:
        print(" - " + domain)
  

Mars: http://www.wikidata.org/entity/Q112
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q11451
Ares: http://www.wikidata.org/entity/Q40901
 - http://www.wikidata.org/entity/Q198
Horus: http://www.wikidata.org/entity/Q84122
 - http://www.wikidata.org/entity/Q527
 - http://www.wikidata.org/entity/Q11573140
Minerva: http://www.wikidata.org/entity/Q126916
 - http://www.wikidata.org/entity/Q37226
 - http://www.wikidata.org/entity/Q49757
 - http://www.wikidata.org/entity/Q1294787
 - http://www.wikidata.org/entity/Q43109919
Astarte: http://www.wikidata.org/entity/Q130274
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q964401
 - http://www.wikidata.org/entity/Q4042967
Seth: http://www.wikidata.org/entity/Q131795
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q79871
 - http://www.wikidata.org/entity/Q124490
 - http://www.wikidata.org/entity/Q182878
 - http://www.wikidata.org/entity/Q473741
 - http://www.wikidata.org/

## Neo4j

In [14]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()
url = os.getenv('NEO4J_BOLT_URL')
user = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(url, auth=(user,password))

In [15]:
# Delete all nodes & relationships
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

In [16]:
# Fetch deities 

sparql.setQuery("""
SELECT  ?deity ?deityLabel ?parentDeity ?parentDeityLabel ?domainOfSaint ?domainOfSaintLabel ?deityOf ?deityOfLabel
WHERE {
  ?deity wdt:P31 ?parentDeity .                   # deites that are an instance of (P31) a parent deity
  ?parentDeity wdt:P279 wd:Q178885  .             # parent deity that is a "subclass of" (P279) 'deity' (Q178885)
  
  ?deity p:P31 ?deity_instance .                  # instance_of (Pax is an instance of a Romain Deity)
  OPTIONAL { ?deity_instance pq:P642 ?deityOf . }      # "of" attribute of the instance_of (Romain Deity OF peace)
   
  OPTIONAL { ?deity wdt:P2925  ?domainOfSaint . } # "domain of saint or deity" (P2925) (Mars is deity of War and Agriculture)

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

with driver.session() as session:  

  for row in results["results"]["bindings"]:
      # print(row["deityLabel"]["value"] + ": " + row["parentDeity"]['value'])  
      
      session.run("MERGE (g:Deity { name: $name, entity: $entity}) RETURN g", 
              name=row["deityLabel"]["value"], 
              entity=row["deity"]['value'])
      
      session.run("MERGE (d:Religion { name: $name, entity: $entity}) RETURN d", 
                  name=row['parentDeityLabel']['value'],
                  entity=row["parentDeity"]['value'])
      
      session.run("MATCH (g2:Deity {entity: $deity_entity}), (d2:Religion {entity: $religion_entity}) MERGE (g2)-[r:PART_OF]->(d2)",
                  deity_entity=row["deity"]['value'],
                  religion_entity=row["parentDeity"]['value'])      
      
      if ("domainOfSaintLabel" in row):
        session.run("MERGE (d:Domain { name: $name, entity: $entity }) RETURN d", 
                name=row["domainOfSaintLabel"]["value"], 
                entity=row["domainOfSaint"]['value'])

        session.run("MATCH (g2:Deity {entity: $deity_entity}), (d2:Domain {entity: $domain_entity}) MERGE (g2)-[r:GOD_OF]->(d2)",
                    deity_entity=row["deity"]['value'],
                    domain_entity=row["domainOfSaint"]['value'])


In [ ]:
# with driver.session() as session:  
#   for row in results["results"]["bindings"]:
#       # We want the "value" attribute of the "comment" field
#       print(row["wargod"]['value'] + "\t" + row["wargodLabel"]["value"])
      
#       session.run("CREATE (g:God { name: $name, entity: $entity}) RETURN g", 
#                   name=row["wargodLabel"]["value"], 
#                   entity=row["wargod"]['value'])
      
#       domains = row['domainsOfSaint']['value'].split(';')    
#       if (domains):
#         for domain in domains:
#             print("\t" + domain)
            
#             session.run("MERGE (d:Domain { entity: $entity}) RETURN d", entity=domain)
            
              
#             session.run("MATCH (g2:God {entity: $god_entity}), (d2:Domain {entity: $domain_entity}) MERGE (g2)-[r:GOD_OF]->(d2)",
#                         god_entity=row["wargod"]['value'],
#                         domain_entity=domain)